In [4]:
from PdfCleaner import PdfCleaner
from os import listdir
from os.path import isfile, join
from PerformanceTesting import PerformanceTester
import pdfplumber
import json
import pandas as pd
import numpy as np
import requests
from pymongo import MongoClient


class Transcripts:
    def __init__(self, ticker):
        self.ticker = ticker
        xinyu = "/Users/xinyuwu/Desktop/fall21/ds3500/DS3500-Final-Project"
        marco = "C:/Users/mtort/Repositories/DS3500-Final-Project"
        emily = "/Users/emilywang/Desktop/DS3500-Final-Project-main-2"
        kelly = "/Users/kelly/Desktop/ds3500/DS3500-Final-Project"
        qi = ""
        self.path = emily + "/transcripts/"+ticker+"_transcripts/"

    def read_files(self):
        return [f for f in listdir(self.path) if isfile(join(self.path, f))]

    def create_dct(self):
        lst_files = self.read_files()
        lst_cleaned = []
        for file in lst_files:
            file_path = self.path + file
            if file_path[-3:] == 'pdf':
                txt = PdfCleaner(file_path)
                date = file[:8]
                # txt_cleaned = txt.clean_stopwords_punctuation()
                txt_cleaned = txt.clean_nums()
                PerfTest = PerformanceTester()
                PerfTest.setTimeframe('day', 1)
                PerfTest.loadArticles([[self.ticker, date, txt_cleaned]])
                try:
                    classification_xy = PerfTest.aquireTargetValues()
                except KeyError:
                    print(f'''Warning: attempted to access market during weekend or after hours
                          Earnings Transcript {file_path} Not Added
                          ''')
                    continue
                dct_cleaned = {'price_change': classification_xy[1][0], 'name': self.ticker, 'date': date, 'transcript': txt_cleaned}
                lst_cleaned.append(dct_cleaned)
        return lst_cleaned

class Database:
    def __init__(self):
        client = MongoClient()
        client.drop_database('transcripts')
        self.db = client.transcripts

    def store_data(self, tickers_lst):
        for t in tickers_lst:
            store = Transcripts(t)
            transcript = store.create_dct()
            self.db.transcript.insert_many(transcript)
            print(t + " transcripts stored successfully")
        return self.db
    

In [5]:
''' To prevent "IOPub data rate exceeded error":
enter into terminal: jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10 '''

tickers = ['AAPL', 'FB', 'MSFT']
    # works: APPL, MSFT
    # doesn't work: FB, GOOGL, NFLX, TSLA, ADBE, CMCSA, COST, AMZN

    # 'APPL', 'MSFT', 'FB', 'GOOGL', 'NFLX', 'TSLA', 'ADBE', 'CMCSA', 'COST', 'AMZN'
    # client = MongoClient()
    # client.drop_database('transcripts')
    # db = client.transcripts


    # for t in tickers:
    #     store = Transcripts(t)
    #     transcript = store.create_dct()
    #     # print(transcript)
    #     db.transcript.insert_many(transcript)
    #     print(t + " transcripts stored successfully")

    # db.transcript.find() # client.transcripts.transcript.find()

database = Database()
db = database.store_data(tickers)
# all_transcripts = db.transcript.find()
# for transcript in all_transcripts:
#     print(transcript)

                          Earnings Transcript /Users/emilywang/Desktop/DS3500-Final-Project-main-2/transcripts/AAPL_transcripts/20161025_Apple_Inc-_Earnings_Call_2016-10-25_FS000000002309526178.pdf Not Added
                          
                          Earnings Transcript /Users/emilywang/Desktop/DS3500-Final-Project-main-2/transcripts/AAPL_transcripts/20161025_Apple_Inc-_Earnings_Call_2016-10-25_FS000000002309526172.pdf Not Added
                          
AAPL transcripts stored successfully
FB transcripts stored successfully


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer


''' Query to test on all APPLE transcripts '''
# apple = db.transcript.find({"name":"APPL"})


''' Query to test on all transcripts '''
all_companies = db.transcript.find()

data = {}
text = []

for transcript in all_companies:
    data[transcript["name"]+" "+transcript["date"]] = transcript["transcript"]
    text.append(transcript["transcript"])
#     print(transcript)
#     df = pd.DataFrame.from_dict(data, orient ='index')




# for name, info in data.items():
''' Use countvectorizer to convert pdf text to a matrix of token counts '''
# vect = CountVectorizer()
# vect.fit(text)
#     print("vocabulary: \n", vect.vocabulary_)


''' Use bag of words to count how many times each word appears in pdf '''
# bag_of_words = vect.transform(text)
#     print(bag_of_words)

    # number of times each unique word appears in this text
# bag_of_words_arr = bag_of_words.toarray()
# #     print(bag_of_words_arr)
# feature_names = vect.get_feature_names()
#     print(feature_names)

''' Display occurrences in dataframe '''
# bow_df = pd.DataFrame(bag_of_words_arr, columns = feature_names)
# bow_df["Transcript (Company)"] = "APPL"
# bow_df.set_index("Transcript (Company & Date)", inplace=True)

''' Use Tfidf vectorizer to match words to TFIDF values '''

vect = TfidfVectorizer(min_df=3, ngram_range = (1, 1)).fit(text)
bag_of_words = vect.transform(text)
feature_names = vect.get_feature_names()
    
tfidf_df = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)
# tfidf_df[name] = text
# tfidf_df.set_index(name, inplace=True)

# tfidf_df.head()
    
    
# tfidfconverter = TfidfTransformer()
# X = tfidfconverter.fit_transform(bag_of_words_arr).toarray()

    
# tfidf_df = pd.DataFrame(X, columns = feature_names)
# tfidf_df["Transcript (Company)"] = "APPL"
# tfidf_df.set_index("Transcript (Company & Date)", inplace=True)

# ''' Show number of occurences of each word in one dataframe and tfidf (weighted) values of each word in another
#     dataframe for each pdf '''
# display(bow_df)
display(tfidf_df)

,aaa,aapl,ab,abb,ability,able,abnormally,about,above,absent,...,ﬂip,ﬂoor,ﬂow,ﬂowing,ﬂows,ﬂuctuate,ﬂuctuation,ﬂuctuations,ﬂuid,ﬂywheel
0,0.0,0.123631,0.000000,0.0,0.003594,0.010291,0.012444,0.012134,0.000000,0.0,...,0.0,0.000000,0.004456,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.114242,0.000000,0.0,0.003321,0.015849,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.004118,0.0,0.000000,0.000000,0.000000,0.006568,0.011499,0.000000
2,0.0,0.063862,0.000000,0.0,0.006684,0.004253,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.062744,0.000000,0.0,0.000000,0.004477,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.122199,0.000000,0.0,0.003553,0.006781,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.004404,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,0.0,0.000000,0.000000,0.0,0.003704,0.001768,0.000000,0.002084,0.000000,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
81,0.0,0.000000,0.000000,0.0,0.003143,0.011998,0.000000,0.003537,0.000000,0.0,...,0.0,0.000000,0.023378,0.0,0.009551,0.000000,0.000000,0.006215,0.000000,0.000000
82,0.0,0.000000,0.000000,0.0,0.017635,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.000000,0.010932,0.0,0.000000,0.010176,0.010767,0.000000,0.000000,0.000000
83,0.0,0.000000,0.010182,0.0,0.006446,0.018455,0.000000,0.000000,0.000000,0.0,...,0.0,0.010182,0.007991,0.0,0.000000,0.000000,0.000000,0.006374,0.000000,0.011158


In [33]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

''' Using PCA to reduce total number of features before feeding into ML model '''
# instantiate the PCA object and request ten components (reduces number of columns/features)
pca = PCA(n_components=10, random_state=3000)


# standardize the features so they are all on the same scale
features_standardized = StandardScaler().fit_transform(bag_of_words.toarray())

# transform the standardized features using the PCA algorithm 
reduced_data = pca.fit_transform(features_standardized)

# show transformed results in dataframe
components = ["Component1", "Component2", "Component3", "Component4", "Component5", "Component6",
             "Component7", "Component8", "Component9", "Component10"]

pca_df = pd.DataFrame(reduced_data)#, columns = components)

''' Obtain target values (whether stock price increased, decreased, or stayed the same) 
from database '''
price_changes = []
all_transcripts = db.transcript.find()
for transcript in all_transcripts:
    price_changes.append(transcript['price_change'])

pca_df['target'] = price_changes 

pca_df



,0,1,2,3,4,5,6,7,8,9,target
0,-14.370612,34.929522,6.866992,0.138975,43.083945,26.666641,-12.637670,2.212028,-0.841859,46.323495,0
1,-10.497373,30.210806,8.287971,-1.123895,7.018501,7.353379,3.098508,4.250223,-11.627328,-6.280092,0
2,-7.212433,23.007934,-14.789445,-2.624524,-15.767124,1.323900,41.466960,0.129794,2.039342,11.192966,1
3,-7.380730,21.070138,-19.801125,-5.100650,8.929719,-0.864220,-8.200650,-4.622732,13.711437,-12.145730,1
4,-11.972317,33.472015,9.580812,2.456007,-13.667796,-0.405706,17.977278,5.968215,-10.963286,-20.793990,0
...,...,...,...,...,...,...,...,...,...,...,...
80,16.662866,-2.316088,-22.087908,-11.311856,-0.363315,-12.725377,3.746636,-2.113741,16.417479,-1.375908,1
81,35.193460,-5.580390,11.409932,-9.951625,1.666313,6.644126,-1.139479,-1.144222,5.156290,-2.746820,0
82,38.160879,-6.107779,-4.026217,70.652309,1.297173,18.620154,2.366126,-11.780831,7.724605,-5.789132,1
83,35.070542,-7.415929,22.115346,-19.827102,-3.290288,16.520365,2.833955,-18.532044,-1.904825,-0.470039,1
